In [10]:
import pickle
import os
from IPython.display import display, Markdown
path = os.path.join(os.path.dirname(os.path.abspath(__vsc_ipynb_file__)), 'processed', 'luong.pkl')
with open('/Users/chungnn/companyproject/processed/luong.pkl', 'rb+') as f:
    content = pickle.load(f)


print(display(Markdown(content[6].page_content)))


Công thức: Lương tháng = Khoản cố định tháng/ số ngày làm việc theo lịch x (Số ngày đi học x 85% + Số ngày đi làm bình thường không đi học x 100%) **(2) Công tác** Lương trong thời gian đi công tác: hưởng 100% tiền lương làm việc thông thường **(3) Phái cử:** Là trường hợp Công ty cử đi làm việc tại một cứ điểm khác trong cùng tập đoàn. Lương và BHXH: theo quy định của tập đoàn _(Quy định điều chuyển công tác quốc tế)_ Bảo hiểm bắt buộc ở Việt Nam: Theo quy định của luật Bảo hiểm xã hội Việt Nam như chi tiết ở Điều 16. **Điều 12. Tiền lương trong thời gian nghỉ** **(1) Nghỉ hưởng lương** Nếu nhân viên nghỉ một trong các loại nghỉ sau: Nghỉ phép, nghỉ hiếu/ hỉ, nghỉ tai nạn lao động, nghỉ bù, nghỉ chu kỳ, nghỉ chế độ con nhỏ, nghỉ trước/ sau khi đi phái cử, sau khi đi công tác, đào tạo, nghỉ làm hộ chiếu theo yêu cầu công tác của Công ty, nghỉ khám nghĩa vụ quân sự, nhân viên vẫn được hưởng nguyên lương những ngày nghỉ này. **(2) Nghỉ hưởng lương từ cơ quan BHXH** Nghỉ hưởng lương từ cơ quan BHXH theo luật Bảo hiểm bao gồm các loại nghỉ sau: nghỉ bản thân ốm, nghỉ con ốm, nghỉ khám thai, nghỉ thai sản, nghỉ dưỡng sức…Những loại nghỉ này được ghi nhận khi có giấy chứng nhận nghỉ việc hưởng Bảo hiểm Xã hội và được tính dựa trên lương đóng BHXH theo Luật Bảo hiểm. **(3) Nghỉ không hưởng lương** Nghỉ không hưởng lương của Công ty bao gồm các loại nghỉ: nghỉ hưởng lương từ BHXH, nghỉ việc riêng, nghỉ việc riêng không lý do. Số tiền bị trừ cho loại nghỉ không hưởng lương = Tiền lương/ số ngày làm việc theo lịch x số ngày nghỉ không hương lương. Trong đó: Tiền lương = Khoản cố định tháng + phụ cấp lương. **Điều 13. Tiền lương đối với các trường hợp nghỉ việc** Đối với trường hợp nghỉ việc, lương của tháng làm việc cuối cùng sẽ được tính dựa trên công thức sau: Lương tháng = Tiền lương / số ngày làm việc theo lịch của tháng x Số ngày làm việc thực tế được tính cho đến ngày làm việc cuối cùng tại Công ty. Ngoài ra, khi người lao động chấm dứt Hợp đồng lao động đúng luật và có thời gian làm việc thường xuyên đủ 12 tháng trở lên thì sẽ được hưởng Trợ cấp thôi việc theo quy định của Luật lao động, mỗi năm được nửa tháng tiền lương Tiền trợ cấp thôi việc = Thời gian làm việc để tính trợ cấp thôi việc x Tiền lương x ½ Trong đó: 7/21

None


In [2]:
os.path.abspath(__vsc_ipynb_file__)

'/Users/chungnn/companyproject/notebooks/exp.ipynb'

In [61]:
import os
import pickle
import faiss
import re
from pathlib import Path
from typing import List
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.retrievers import EnsembleRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_pymupdf4llm import PyMuPDF4LLMLoader
from langchain_community.retrievers import BM25Retriever
from langchain.docstore import InMemoryDocstore 
from langchain_community.document_loaders.parsers.images import TesseractBlobParser
from langdetect import detect, LangDetectException
from langchain.retrievers.multi_query import MultiQueryRetriever    
from langchain_core.output_parsers import BaseOutputParser
from pydantic import BaseModel, Field
from langgraph.graph import START, END
from uuid import uuid4
import logging
from IPython.display import Markdown, display
from underthesea import word_tokenize
print('import completed')

PDF_PATH = 'true_test.pdf'
FAISS_INDEX_PATH =  'faiss_index'
IMAGE_DIR = 'extracted_images'
EMBEDDING_MODEL_NAME = 'mxbai-embed-large:335m'
EMBEDDING_MODEL2 = 'vinai/phobert-base-v2'
MODEL_NAME = 'llama3.1:8b'
CHUNKS_CACHE_PATH = "chunks_cache2.pkl"

import completed


In [12]:
loader = PyMuPDF4LLMLoader(PDF_PATH, mode = 'page' , extract_images=True, images_parser=TesseractBlobParser(langs=['vie', 'jpn']), table_strategy='lines')
print('loader created')

loader created


In [7]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [3]:
VIET_PATTERN = re.compile(r'[\w\sÀ-ỹ\u0300-\u036F.,;:!?()“”‘’\-–—]+', re.UNICODE)
JAP_PATTERN = re.compile(r"[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+") #Hiragana, Katakana, Kanji

In [70]:
def detect_language_with_fallback(text, fallback='vi'):
    if len(text.strip()) < 50:  # Too short → unreliable detection
        return fallback
    try:
        lang = detect(text)
        if lang in ['vi', 'ja']:
            return lang
        else:
            return fallback
    except LangDetectException:
        return fallback
def preprocess_vie(text):
    text = JAP_PATTERN.sub('', text)
    vie_chars = "ạảãàáâậầấẩẫăắằặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ"
    pattern = re.compile(f"([{vie_chars}]) ([{vie_chars}])")
    while True:
        new_text = pattern.sub(r"\1\2", text)  
        if new_text == text:
            break
        text = new_text
    try:
        tokens = word_tokenize(text)
        text = " ".join(tokens)
    except:
        pass
    text = re.sub(r"\s+", ' ', text).strip()
    return text
def preprocess_jpn(text):
    VIET_LATIN_PATTERN = re.compile(r'[a-zA-ZÀ-ỹ\u0300-\u036F]+')
    text = VIET_LATIN_PATTERN.sub('', text)
    JP_KEEP_PATTERN = re.compile(r'[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF。、・「」『』（）！？…―\s]')
    text = ''.join(char for char in text if JP_KEEP_PATTERN.match(char))
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def load_and_chunk_from_document(path):
    loader = PyMuPDF4LLMLoader(
        path, 
        mode='page', 
        extract_images=True,
        images_parser=TesseractBlobParser(langs=['vie', 'jpn']),
        table_strategy='lines'
    )

    docs = loader.lazy_load()
    vietnamese_docs, japanese_docs = [], []

    for doc in docs:
        print(f'Processing document: {doc.metadata.get("source", "unknown source")}, page {doc.metadata.get("page", "N/A")}')
        content = doc.page_content
        lang = detect_language_with_fallback(content, fallback='vi')

        if lang == 'vi':
            cleaned = preprocess_vie(content)
            if len(cleaned) > 10:  
                vietnamese_docs.append(
                    Document(
                        page_content=cleaned,
                        metadata={**doc.metadata, 'language': 'vi'}
                    )
                )
        elif lang == 'ja':
            cleaned = preprocess_jpn(content)
            if len(cleaned) > 10:
                japanese_docs.append(
                    Document(
                        page_content=cleaned,
                        metadata={**doc.metadata, 'language': 'ja'}
                    )
                )

    return vietnamese_docs, japanese_docs



In [ ]:
class HierarichicalChunker:
    def __init__(self, vector_store, embedding_fn):
        self.vector_store = vector_store
        self.embedding_fn = embedding_fn
        self.parent_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 1200,
            chunk_overlap = 200,
            separators = ['\n', '.', ' ', ''],
            keep_separator = True
        )
        self.child_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 500,
            chunk_overlap = 50,
            separators = ['\n', '.', ' ', ''],
            keep_separator = True
        )
    def chunk(self, documents: List[Document]) -> List[Document]:
        parent_chunks, child_chunks = [], []

        for doc in documents:
            p_docs = self.parent_splitter.split_documents([doc])
            for p_doc in p_docs:
                parent_id = str(uuid4())
                p_doc.metadata['chunk_level'] = 'parent'
                parent_chunks.append(p_doc)

                child_docs = self.child_splitter.split_documents([p_doc])
                for c_doc in child_docs:
                    c_doc.metadata['chunk_level'] = 'child'
                    c_doc.metadata['parent_id'] = parent_id
                    child_chunks.append(c_doc)
        return {
            'parent_chunks': parent_chunks,
            'child_chunks': child_chunks
        }
    def add_to_vector_store(self, chunks: List[Document]):
        for chunk in chunks['parent_chunks']:
            self.vector_store.add_documents([chunk])
        for chunk in chunks['child_chunks']:
            self.vector_store.add_documents([chunk])
    


In [77]:
vietnames_documents, japanese_documents = load_and_chunk_from_document(PDF_PATH)


Processing document: true_test.pdf, page 0
Processing document: true_test.pdf, page 1
Processing document: true_test.pdf, page 2
Processing document: true_test.pdf, page 3
Processing document: true_test.pdf, page 4
Processing document: true_test.pdf, page 5
Processing document: true_test.pdf, page 6
Processing document: true_test.pdf, page 7
Processing document: true_test.pdf, page 8
Processing document: true_test.pdf, page 9
Processing document: true_test.pdf, page 10
Processing document: true_test.pdf, page 11
Processing document: true_test.pdf, page 12
Processing document: true_test.pdf, page 13
Processing document: true_test.pdf, page 14
Processing document: true_test.pdf, page 15
Processing document: true_test.pdf, page 16
Processing document: true_test.pdf, page 17
Processing document: true_test.pdf, page 18
Processing document: true_test.pdf, page 19
Processing document: true_test.pdf, page 20
Processing document: true_test.pdf, page 21
Processing document: true_test.pdf, page 2

In [30]:
vn_embedding_fn = HuggingFaceEmbeddings(
    model_name='intfloat/multilingual-e5-large',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
jp_embedding_fn = HuggingFaceEmbeddings(
    model_name = 'intfloat/multilingual-e5-small',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print('embedding functions created')

C:\Users\vnintern02\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vnintern02\.cache\huggingface\hub\models--intfloat--multilingual-e5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Sto

embedding functions created


In [ ]:
vector_database_vn = FAISS(embedding_function=vn_embedding_fn)
vector_database_jp = FAISS(embedding_function=jp_embedding_fn)
chunker_vn = HierarichicalChunker(vector_database_vn)
chunker_jp = HierarichicalChunker(vector_database_jp)
vn_chunks = chunker_vn.chunk(vietnames_documents)
jp_chunks = chunker_jp.chunk(japanese_documents)

TypeError: FAISS.__init__() missing 3 required positional arguments: 'index', 'docstore', and 'index_to_docstore_id'

In [37]:
if vietnamese_chunks:
    vn_vector_store = FAISS.from_documents(vietnamese_chunks, vn_embedding_fn)
    # print(f'Vietnamese vector store created and saved at {FAISS_INDEX_PATH}_vn.pkl.')
if japanese_chunks:
    jp_vector_store = FAISS.from_documents(japanese_chunks, jp_embedding_fn)
    # with open(f'{FAISS_INDEX_PATH}_jp.pkl', 'wb') as f:
    #     pickle.dump(jp_vector_store, f)
    # print(f'Japanese vector store created and saved at {FAISS_INDEX_PATH}_jp.pkl.')

In [ ]:
bm25_vn_retriever = BM25Retriever.from_documents(
    vietnamese_chunks,
    preprocess_func=word_tokenize,
    search_kwargs={'k':5}
)

In [39]:
bm25_jp_retriever = BM25Retriever.from_documents(
    japanese_chunks,
    search_kwargs={'k':5}
)

In [40]:
vn_retriever = vn_vector_store.as_retriever(search_kwargs={'k':20 , 'filter' : {'language': 'vi'}})
jp_retriever = jp_vector_store.as_retriever(search_kwargs={'k':5})
unified_retriever = EnsembleRetriever(
    retrievers = [vn_retriever, jp_retriever],
    weights = [0.9, 0.1]
)

In [51]:
vn_ensemble_retriever = EnsembleRetriever(
    retrievers = [vn_retriever, bm25_vn_retriever],
    weights = [0.6, 0.4]
)

In [42]:
question = 'Giờ làm việc cơ bản?'
retrieved_docs = vn_ensemble_retriever.invoke(question)

In [43]:
for i in range(len(retrieved_docs)):
    display(Markdown(retrieved_docs[i].page_content))

! [ Công ty TNHH DENSO Việt Nam - Việc chụp ảnh trong Công ty phải được thực hiện bằng máy ảnh của Công ty với mục đích công việc vàđược phê duyệt với từng mức độ bảo mật ( đeo băng tay vải màu cam và có dòng chữ “ Camera ” ) ; - Ảnh chụp trong Công ty được coi là tài sản thông tin của Công ty . Do vậy , cầm mọi hình thức phân phối ảnh chụp trong Công ty vàảnh chụp cá nhân mặc đồng phục DENSO ra bên ngoài , bao gồm cả trang mạng xã hội nếu chưa được phê duyệt bởi các cấp có thâm quyên ; - Công ty thực hiện đào tạo người lao động về hoạt động bảo vệ tài sản , bí mật kinh doanh công nghệ của Công ty . 3 . Xử lí kỷ luật Đối với những hành vi xâm phạm tài sản , bí mat kinh doanh công nghệ của Công ty , tùy theo lỗi , mức độ nghiêm trọng về hậu quả mà có thể bị xử lý kỷ luật theo điều 39 chương 7 hoặc Công ty khởi kiện theo quy định của pháp luật . CHƯƠNG 4 - NHIỆM VỤ LÀM VIỆC Phần 1 : Giờ làm việc Điều 11 : Giờ làm việc cơ bản 1 . Lịch làm việc hàng năm ( bao gồm ngày nghỉ tuần ) được quy định cụ thể và thông báo cho người lao động vào đầu năm sau khi tham khảo ý kiến của tổ chức đại diện người lao động . 2 . Đối với nhân viên khối văn phòng : 2.1 Giờ làm việc bình thường không quá 10 giờ / ngày và 48 giờ / tuần . 2.2 Ngày nghỉ hằng tuần của nhân viên là Thứ Bảy , Chủ Nhật 2.3 Giờ làm việc như sau : GIỜ LÀM VIỆC THỰC TẾ NGHỈ NGẮN ( 8 Giờ 35 Phút ) ( 20 phút ) h Nghỉ giải lao và so zÄ 。 Buổi sáng ïn trữ Buổi chiều Nghỉ ngắn § : 10 ~ 12 : 00 | 12 : 00 ~ 12 : 30 12 : 30 ~ 17 : 15 | 20 phút ( thời gian linh hoạt ) 3 . Đối với nhân viên khối nhà máy : 3.1 Ngày nghỉ hằng tuần của nhân viên là Chủ Nhật . 3.2 Về cơ bản , giờ của các ca làm việc sẽ như sau : Ca | Bấtđầu | Giảilaoldnl | Giải lao lần 2 | ( ; ; 1 olận4 | Kếtthúc và Ăn giữa ca Cai | 06 : 00 | 08 : 00 < 08 : 10 | 10 : 00 ~ 1030 | 12 : 15 ~ 1225 | 14 : 00 Ca2 | 14 : 00 | 16 : 00 ~ 16 : 10 | 18 : 00 ~ 18 : 30 | 20 : 15 ~ 20 : 25 | 22 : 00 Ca3 | 22 : 00 _ | 00 : 00 ~ 00 : 10 | 02 : 00 ~ 02 : 45 | 04 : 30 ~ 04 : 40 | 06 : 00 4 . Giờ nghỉăn giữa ca : ca ngày là 30 phút , ca đêm là 45 phút . ] ( # )

! [ Công ty TNHH DENSO Việt Nam TIẾNG VIỆT NỘI QUY LAO ĐỘNG MỤC LỤC CHUONG 1 : NHŨNG QUY ĐỊNH CHUNG Trang : 4 Điều 1 : Mục đích của nội quy Trang : 4 Điều 2 : Phạm vi áp dụng Trang : 4 Điều 3 : Người lao động Trang : 4 Điều 4 : Cấp trên Trang : 4 Điều 5 : Người sử dụng lao động Trang : 4 CHƯƠNG 2 : TRẬT TỰTRONG CÔNG TY Trang : 4 Điều 6 : Thái độ cơ bản Trang : 4 Điều 7 : Chấp hành nội quy Trang : 4 Điều 8 : Ra vào công ty Trang : 5 CHƯƠNG 3 : CHUYỂN NGƯỜI LAO ĐỘNG LÀM Trang : 6 CÔNG VIỆC KHÁC VỚI HỢP ĐỒNG LAO ĐỘNG . BẢO VỆ BÍ MẬT KINH DOANH , CÔNG NGHỆ , SỞ HỮU TRÍ TUỆ CỦA CÔNG TY Điều 9 : Thay đổi công việc khác với Hợp đồng lao động Trang : 6 Điều 10 : Bảo vệ bí mật kinh doanh , công nghệ của công ty Trang : 7 CHƯƠNG 4 : NHIỆM VỤ LÀM VIỆC Trang : 8 Phần 1 : Giờ làm việc Trang : 8 Điều 11 : Giờ làm việc cơ bản Trang : 8 Điều 12 : Lam thêm giờ Trang : 9 Điều 13 : Ngoại lệ trong trường hợp khẩn cấp Trang : 10 Phần 2 : Ngày nghỉ Trang : 10 Điều 14 : Nghỉ lễ Trang : 10 Điều 15 : Nghỉ phép Trang : 10 ] ( # )

! [ Công ty TNHH DENSO Việt Nam Điều 16 : Thủ tục xin nghỉ phép Trang : 11 Điều 17 : Xem xét lại Trang : 11 Điều 18 : Các ngày nghỉ khác Trang : 11 Điều 19 : Nghỉ không lương Trang : 11 Điều 20 : Thủ tục xin nghỉốm Trang : 12 Điều 21 : Nghỉ không phép Trang : 12 Điều 22 : Đi làm muộn Trang : 12 Phần 3 : Đi công tác Trang : 13 Điều 23 : Đi công tác Trang : 13 CHUONG 5 : SỨC KHỎE VA AN TOAN Trang : 13 Phân 1 : Thái độ cơ bản Trang : 13 Điều 24 : Thái độ cơ bản Trang : 13 Phần 2 : An toàn Trang : 13 Điều 25 : Trách nhiệm của Công ty Trang : 13 Điều 26 : Trách nhiệm của Người lao động Trang : 14 Điều 27 : Sử dụng trang thiết bị Trang : 14 Điều 28 : Hạn chếđối với phụ nữ và một số lao động khác Trang : 14 Điều 29 : Báo cáo vàđiều tra về tai nạn có thể xảy ra Trang : 14 Phân 3 : Sức khỏe Trang : 15 Điều 30 : Quy định về sức khỏe Trang : 15 Điều 31 : Kiểm tra sức khỏe Trang : l5 Điều 32 : Nghiêm cấm đến nơi làm việc Trang : 15 Điều 33 : Sự ngăn chặn những bệnh truyền nhiễm Trang : l5 CHƯƠNG 6 : PHÒNG CHÓNG QUÁY RÓI TÌNH Trang : 16 DỤC TẠI NƠI LÀM VIỆC Điều 34 : Định nghĩa về “ Quay rối tình dục tại nơi làm việc ” Trang : 16 Điều 35 : Quyền và nghĩa vụ của người lao động về Trang : l6 Phòng chồng quấy rối tình dục tại nơi làm việc Điều 36 : Quy định về phòng chống quấy rồi tình dục Trang : 16 tại nơi làm việc ] ( # )

! [ Công ty TNHH DENSO Việt Nam 9 . - Nhân viên có thời gian làm việc trong khoảng từ 22 : 00 đến 6 : 00 sáng ngày hôm sau được hưởng phụ cấp làm ca đêm bằng 35 % tiền lương của công việc làm ban ngày . . Thời điểm bất đầu hoặc kết thúc của giờ làm việc được quy định ở trên có nghĩa làđã “ sắn sàng làm việc ” hoặc “ kết thúc công việc ” tại vị trí làm việc . Thời gian đi tới nơi làm việc , thay trang phục , giao ca ... và từ nơi làm việc về không được tính là thời gian làm việc . . Việc không thể bắt đầu công việc vào thời điểm bắt đầu giờ làm việc hoặc ca làm việc mà không có lí do chính đáng sẽ bị coi là “ Di muộn ” . Rời vị trí làm việc mE \ \ ] không được cấp trên cho phép trước thời điểm kết thúc giờ làm việc sẽ bị coi là “ Ve sớm ” . h TB hợp bất khả kháng như dịch bệnh , sự cốđiện nước , thiên tai , cháy .. người lao động có thểđược yêu cầu nghỉ hàng tuần vào bất kỳ ngày nào trong tuần , không cốđịnh như trên lịch và Công ty sẽ thông báo trước cho người lao động bằng văn bản . Giờ làm việc trong thời gian công tác nước ngoài được quy định riêng . Nàng 12 : Làm thêm giờ bai . Khi cần làm thêm giờ do yêu cầu sản xuất kinh doanh , Công ty là người thỏa thuận , đồng ý cho người lao động làm thêm giờ khi đáp ứng các điều kiện sau : - Phai được sựđồng ý của người lao động tham gia làm thêm về các nội dung sau đây : thời gian làm thêm , địa điểm làm thêm , công việc làm thêm ; - Bảo đảm tổng số giờ làm việc bình thường và số giờ làm thêm của người lao động không quá 12 h / ngay : Tổng số giờ làm thêm không quá 40 h / tháng , không quá 200 h / năm ; Khi làm thêm từ 200 h đến 300 h thì công ty sẽ thực hiện các bước theo quy định của pháp luật . . Làm thêm giờ trong trường hợp đặc biệt Công ty có quyên yêu cầu người lao động làm thêm giờ vào bắt kỳ ngày nào mà không bị giới hạn về số giờ làm thêm theo quy định và người lao động không được từ chối trong trường hợp sau đây : - Thực hiện lệnh động viên , huy động đảm bảo nhiệm vụ quốc phòng , an ninh theo quy định của pháp luật ; Thực hiện các công việc nhằm bảo vệ tính mạng con người , tài sản của cơ quan , tổ chức , cá nhân trong phòng ngừa , khắc phục hậu quả thiên tai , hỏa hoạn , dịch bệnh nguy hiểm và thảm họa , trừ trường hợp có nguy cơ ảnh hưởng đến . tính mạng , sức khỏe của người lao động theo quy định của pháp luật về an toàn , vệ sinh lao động . Người lao động làm thêm giờđược trả lương như sau : 3.1 Vào ngày làm việc bình thường : bằng 150 % của tiền lương giờ trong ngày làm việc bình thường ; 3.2 Vào ngày nghỉ tuần : bằng 200 % của tiền lương giờ trong ngày làm việc bình thường ; 3.3 Vào ngày nghỉ lễ : bằng 300 % của tiền lương giờ trong ngày làm việc bình thường . - 9 - ] ( # )

! [ Công ty TNHH DENSO Việt Nam 3.4 Người lao động làm thêm giờ vào ban đêm thì ngoài việc trả lương theo quy định tại khoản 3.1 , 3.2 và 3.3 Điều này , người lao động còn được trả thêm 20 % tiền lương theo tiền lương giờ làm việc ban ngày . Điều 13 : Ngoại lệ trong trường hợp khẩn cấp Khi có những nhu cầu tạm thời do ảnh hưởng nghiêm trọng của thiên tai , hành động phá hoại của kẻ thù , hỏa hoạn hoặc bệnh dịch mà Công ty phải giải quyết , Công ty có thể yêu cầu người lao động làm thêm giờ quá 4 tiếng / ngày . Công ty sẽ có báo cáo với cơ quan quản lý lao động cấp trên trong những trường hợp này . Phần 2 : Ngày nghỉ Điều 14 : Nghỉ lễ 1 . Người lao động được nghỉ làm việc , hưởng nguyên lương những ngày lễ sau : - Tết dương lịch : 1 ngày ( ngày 1 tháng 1 dương lịch ) - Tết âm lịch : 5 ngày - Ngày Chiến thắng : 1 ngày ( ngày 30 tháng 4 dương lịch ) - Ngày Quốc tế lao động : 1 ngày ( ngày 1 tháng 5 dương lịch ) - Ngày Quốc khánh : 2 ngày ( ngày 2 tháng 9 dương lịch và 1 ngày trước hoặc sau ngày Quốc Khánh ) - Ngày Giỗ Tổ Hùng Vương : 1 ngày ( ngày 10 tháng 3 âm lịch ) 2 . Nếu bất kỳ ngày lễ nào nói trên trùng vào ngày nghỉ tuần , người lao động sẽ được nghỉ bù vào ngày tiếp theo . 3 . Công ty sẽ thông báo Lịch làm việc cụ thể của năm tiếp theo cho toàn thể nhân viên vào cuối năm trước sau khi đã tham khảo ý kiến của Tổ chức đại diện người lao động trên cơ sở tuân thủ thời giờ làm việc , thời giờ nghỉ ngơi của Bộ luật lao động . Điều 15 : Nghỉ phép 1 . Cách tính ngày phép : 1.1 Thời gian thử việc và tập nghềđược coi là thời gian làm việc của người lao động để tính số ngày phép . 1.2 Người lao động làm đủ 12 tháng / năm sẽđược hưởng 12 ngày phép . Số ngày nghỉđược tăng theo thâm niên làm việc nhưng không vượt quá 20 ngày / năm , cụ thể như sau : s _ Cứ 4 năm được hưởng thêm 1 ngày s _ Từ năm thứ 8 , cứ 3 năm được hưởng thêm 1 ngày * _ Từ năm thứ 14 , cứ 2 năm được hưởng thêm 1 ngày * _ Từ năm thứ 20 , cứ 1 năm được hưởng thêm 1 ngày 1.3 Người lao động được tuyển dụng trong năm được hưởng số ngày phép như sau : * Nenu vào làm việc trước ngày 16 của tháng thì sẽđược tính phép . ngay từ tháng đó ; © Neu vào làm việc từ ngày 16 của tháng trởđi thì sẽđược tính phép từ tháng tiếp theo . = lỦ - ~ ] ( # )

...

! [ Công ty TNHH DENSO Việt Nam 1.2 . Nghiêm chỉnh chấp hành hướng dẫn và yêu cầu của cấp trên ; không được rời khỏi nơi làm việc nếu không được cấp trên cho phép ; 1.3 . Nghiêm chỉnh chấp hành quy định về việc mặc đồng phục ( trừ những trường hợp Công ty có yêu cầu đặc biệt riêng ) ; 1.4 . Nghiêm chỉnh chấp hành các nguyên tắc làm việc và báo cáo với cấp trên hoặc đồng nghiệp có liên quan khi cần ; 1.5 . Giữ gìn tài sản , thiết bị , sản phẩm và vật liệu của Công ty ; 1.6 . Không được tiếp khách riêng trong giờ làm việc nếu không được cấp trên cho phép ; 1.7 . Sử dụng mọi thông tin , tài liệu và các dữ kiện khác liên quan đến công việc chỉđể phục vụ cho mục đích của công việc được giao và không tiết lộ các thông tin , tài liệu , đữ kiện một cách trực tiếp hoặc gián tiếp cho một bên thứ ba nếu không được Công ty cho phép ; 1.8 . Không lạm dụng phạm vi trách nhiệm của mình ; 1.9 . Không cho phép các hoạt động bên ngoài ảnh hưởng đến trách nhiệm hoặc công việc được giao ; 1.10 . Không gây ôn ào hoặc làm ảnh hưởng đến trật tự và sự yên tĩnh trong Công ty ; 1.11 . Không nói xấu , làm hại đến thanh danh và uy tín của Công ty hoặc các cá nhân làm việc cho Công ty ; 1.12 . Không thực hiện hoặc tham gia vào các hoạt động phát thanh , xuất bản , phân phối , lưu thông hoặc trưng bày các ấn phẩm văn hóa hoặc tranh ảnh hoặc các hành động tương tự trong Công ty không liên quan đến công việc mà không được Công ty cho phép ; 1.13 . Không tham dự vào vấn đề tiên bạc hoặc buôn bán kiếm lời trong Công ty ; 1.14 . Không bắt buộc người khác quyên góp tiền hay ký tên nhằm mục đích cá nhân ; trục lợi cho bản thân ; làm trái quy định của Công ty ; 1.15 . Không vào những khu vực cấm nếu không phải là nơi làm việc của mình hoặc sử dụng tài sản của Công ty nếu không được Công ty cho phép ; 1.16 . Không sử dụng tài sản của Công ty vào mục đích riêng , sửa chữa hoặc sản xuất tài sản cá nhân bằng các trang thiết bị / tài sản của Công ty hoặc những việc khác tương tự ; 1.17 . Không lạm dụng chức vụ , quyền hạn trong công việc để tìm kiếm lợi nhuận , ưu đãi cá nhân hoặc trao cho người ngoài Công ty ; 1.18 . Không nhận hoặc đồi hỏi các khoản tiền bất chính ; 1.19 . Không tham gia các hoạt động phạm pháp ; 1.20 . Không được quấy rối tình dục tại nơi làm việc theo quy định tại Chương 6 Nội quy lao động này . 2 . Người lao động phải tuân thủ luật giao thông , đảm bảo an toàn giao thông . Điều 8 : Ra vào công ty 1 . Về việc ra vào nơi làm việc , mọi người lao động phải : ] ( # )

! [ Công ty TNHH DENSO Việt Nam 3 . Nguy cơ tai nạn nếu được phát hiện cũng phải được báo cáo ngay cho các phòng ban chức năng và Phòng Nhân sựđể cóđối sách kịp thời . Phần 3 : Sức khỏe Điệu 20 30 : Quy định về sức khỏe . Người lao động ( bên cạnh các yêu cầu khác ) phải có xác nhận đủ sức khỏe do bác sĩ của bệnh viện mà Công ty chỉđịnh cấp thì mới được nhận vào làm tại Công ty . Chi phí khám sức khỏe ban đầu do Công ty chỉ trả . . Người lao động phải tự bảo vệ sức khỏe của mình để có thể hoàn thành tốt công việc . - Người lao động xin nghỉ vì lý do sức khỏe , phải có xác nhận của cơ quan y tế . Nếu người lao động có chứng từ hợp lệ từ cơ sở y tế , người đó sẽđược hưởng chếđộ bảo hiểm theo quy định của cơ quan bảo hiểm . Điều 31 : Kiểm tra sức khỏe 1 . Người lao động ngoài việc phải có giấy chứng nhận sức khỏe trước khi được tuyển dụng vào Công ty , sẽđược định kỳ khám sức khỏe hàng năm . Thời gian khám và số lần khám tùy theo vị trí công việc , do Công ty bố trí . Chi phí khám sức khỏe định kỳ do Công ty chỉ trả . . Nếu người lao động không thể khám sức khỏe vào ngày do Công ty bố trí , người đó phải báo cáo lý do cho Phụ trách Bộ phận và Phòng chức năng . Nếu lý do này được chấp nhận , người lao động sẽđược kiểm tra sức khỏe vào ngày khác và Công ty sẽ trả 100 % chỉ phí . Nếu không được chấp nhận , người lao động sẽ phải tự chi trả phí khám sức khỏe và nộp kết quả cho Công ty . Điều 32 : Nghiêm cấm đến nơi làm việc Người lao động bị nghiêm cấm đến nơi làm việc nếu họ bị bất kỳ bệnh nào trong những bệnh dưới đây : 1 „ 2 。 5 . 4 . Bệnh thương hàn , bệnh sốt phát ban , bệnh bạch hầu , bệnh tả , bệnh sởi , bệnh ho gà , bệnh uốn ván ; Chứng loạn thân kinh , động kinh hay những bệnh thần kinh khác ảnh hưởng đến công việc của họ ; Những người nhiễm bệnh xã hội hoặc bệnh lây ngoài da ; Các bệnh dễ lây lan khác như cúm , đau mắt đỏ , ... gây ảnh hưởng đến công việc hoặc đến những người cùng làm việc . Điều 33 : Sự ngăn chặn những bệnh truyền nhiễm 1 . Khi người lao động mắc bệnh truyền nhiễm hoặc sống cùng người bị mắc bệnh truyền nhiễm hoặc nghỉ có sự ngộđộc thức ăn ... thì phải báo cáo ngay cho Công ty thông qua cấp trên trực tiếp để có sự chỉ dẫn hoặc cấp cứu kịp thời . - 15 - ] ( # )

! [ Công ty TNHH DENSO Việt Nam Bản Nội quy Công ty có hiệu lực ngay sau thời hạn 15 ngày kể từ ngày Ban quản lý các Khu công nghiệp và Khu chế xuất Hà Nội nhận được hồ sơđăng ký . Công ty TNHH DENSO Việt Nam Tổng Giám đốc 28 = ] ( # )

! [ Công ty TNHH DENSO Việt Nam s Dùng tài sản / tiền của Công ty vào mục đích cá nhân hoặc dùng một cách bất hợp pháp , nhận hối lộ từ bất kỳ cơ quan , tổ chức hoặc cá nhân có liên quan đến hoạt động sản xuất , kinh doanh , vận hành của Công ty đưới bất kỳ hình thức nào ; e_ Đánh nhau , tấn công , ham dọa hoặc xi giuc việc tấn công người khác trong phạm vi Công ty gây thiệt hại đặc biệt nghiêm trọng về tài sản và lợi ích Công ty hoặc gây ảnh hưởng nghiêm trọng tới trật tự an ninh trong Công ty ; < Giả mạo giấy tờ , chữ ký , thẻ nhân viên hoặc bất kỳ những thứ khác tương tựđể dùng hoặc cho người khác mượn nhằm trục lợi cá nhân ; 3.2 Người lao động bị xử lý kỷ luật kéo dài thời hạn điều chỉnh lương mà tái phạm trong thời gian chưa được xóa kỷ luật hoặc bị xử lý kỷ luật cách chức mà tái phạm ; 3.3 Người lao động tựý bỏ việc 05 ngày cộng dồn trong thời hạn 30 ngày hoặc 20 ngày cộng dồn trong thời hạn 365 ngày tính từ ngày đầu tiên tựý bỏ việc mà không có lý do chính đáng ; 3.4 Làm tổn hại đến danh dự và tín nhiệm của Công ty một cách nghiêm trọng : vu khống , bôi nhọ , thực hiện các hành vi làm ảnh hưởng đến hình ảnh Công ty ... trước các cơ quan , tổ chức hoặc báo chí , truyền thông ; 3.5 Có bằng chứng về việc thực hiện hành vi quấy rối tình dục của nhân viên tại nơi làm việc , cụ thể như sau : © _ Hành động , cử chỉ có tính chất tình dục như : vuốt ve , cấu véo , ôm ấp hay hôn cho tới tắn công tình dục , cưỡng dâm , hiếp đâm . < Đề nghị , yêu cầu , gợi ý trao đổi quan hệ tình dục lấy sựđánh giá ưu ái hoặc sự hứa hẹn công việc , lương , thưởng . Phạm vi nơi làm việc được hiểu là phạm vi trong Công ty và nơi người lao động được cửđên làm việc / công tác , học tập . Điều 40 : Hoàn cảnh làm giảm nhẹ lỗi Nếu có những hoàn cảnh làm giảm nhẹ lỗi thì mức độ kỷ luật đối với người lao động sẽđược giảm nếu Công ty xét thấy thích hợp . Điều 41 : Trách nhiệm vật chất 1 . Khi người lao động làm hư hỏng dụng cụ , thiết bị hoặc có hành vi khác gây thiệt hại cho tài sản của doanh nghiệp thì phải bồi thường theo quy định của pháp luật về thiệt hại đã gây ra . Nếu gây thiệt hại không nghiêm trọng do sơ xuất với giá trị không quá 10 tháng lương tối thiểu vùng do Chính phủ quy định thì phải bồi thường nhiều nhất là 03 tháng lương và bị khấu trừ dân vào lương ( không quá 30 % tiền lương hàng tháng ) . 2 . Người lao động làm mất dụng cụ , thiết bị , làm mất các tài sản khác do doanh nghiệp giao hoặc tiêu hao vật tư quáđịnh mức cho phép thì tùy từng = Đi = ] ( # )

! [ Công ty TNHH DENSO Việt Nam Bước Đại diện . Đại diện của . Phòng chức năng . Cấp trên của Nhân viên Người làm | Công ty Tố chức đại nhân viên vi phạm . vipham | chứng ( nếu có ) | ? : Họp diện NB | Xem - Ne sạn Sim RI Phòng Legal | MGR trở lên xét kỷ Vai trò - Đưa ra chỉ | - Bảo vệ - Tổ chức cuộc hợp xem xét _ | Kiểm soát việc phòng tránhtái | - gmoco | Đưara bằng trong cuộc | đạo quyền lợi kỷ luật ( gửi thư mời ít nhất | diễn trong tương lai -> Vì vậy vềhành | chứng ( nếu có ) | luật họpxem | - TổngGiám | hợppháp 5 ngày làm việc trước khi | cấp trên của người vi phạm . ww * xét kỷ luật | đốclà của người cuộc họp diễn ra ) „ .. | thực hiện báo cáo chỉ tiết các phạm người lao động ~ Tống hợp các hình thức xử nội dung sau : ( theo . quyết định . lý kỷ luật với các hành vi > Xác nhận diễn biến sự việc từng hình thức . tương tự trong quá khứ — | > Đào tạo trường xử lý kỷ luật - Đề xuất hình thức xử lý kỷ | > Phỏng vấn người vi phạm . hợp ) luật > Các biện pháp phòng tránh _ | - Cam kết tái diễn . Cuộc họp xem xét kỷ luật sẽđược lập thành biên bản và cóđầy đủ chữ ký xác nhận của thành viên tham dự . Trường hợp một trong các thành viên đã tham dự cuộc họp mà không ký vào biên bản thì phải ghi rõ lý do . - Trường hợp nhân viên vi phạm không có mặt theo như thư mời , Công ty sẽ tiến hành họp xem xét kỷ luật vắng mặt . - ( * ) Tùy từng trường hợp , Công ty sẽ không tổ chức họp xem xét kỷ luật , chỉ dừng lại ở mục phỏng van / thao luận ... với Ban Giám đốc . Bước | Ngoại trừ hình thức kỷ luật khiến trách miệng , đối với các hình thức kỷ luật 3 : Ban | khác , Công ty sẽ ban hành Quyết định ký luật ( do Tổng Giám đốc ký phê hành | duyệt ) . quyết định kỷ luật Bước | Phòng chức năng ( Phòng Legal ) lưu giữ hô sơ xử lý kỷ luật lao động và thực 4 : Lưu | hiện các công việc liên quan khác . hồ sơ Điều 39 : Các hình thức kỷ luật Công ty sẽáp dụng một trong các hình thức kỷ luật sau đây đối với người lao động vi phạm kỷ luật lao động , tùy thuộc vào mức độ nghiêm trọng của hành vi vi phạm . Khiển trách Kéo dài thời hạn điều chỉnh lương không quá 6 tháng hoặc cách chức Sa thải 1 . Khiển trách : 1.1 Khiển trách miệng 1.2 Khiển trách bằng văn bản : Khi công ty xét thấy người lao động phạm vào một trong những việc sau , người đó sẽ bị khiển trách bằng văn bản : 1.2.1 Vi phạm quy định trong Nội quy lao động của Công ty mà không gây hậu quả ( trừ trường hợp vi phạm các điều kiện đề kỷ luật sa thải ) ; sl8s ] ( # )

In [22]:
%pip install FlagEmbedding

  Using cached flagembedding-1.3.5-py3-none-any.whl
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.17.1-py3-none-any.whl.metadata (14 kB)
  Using cached ir_datasets-0.5.11-py3-none-any.whl.metadata (12 kB)
  Using cached sentencepiece-0.2.1-cp311-cp311-win_amd64.whl.metadata (10 kB)
  Using cached pyarrow-21.0.0-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.3.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached beautifulsoup4-4.13.5-py3-none-any.whl.metadata (3.8 kB)
  Using cached inscriptis-2.6.0-py3-none-any.whl.metadata (25 kB)
  Using cached lxml-6.0.1-cp311-cp311-win_amd64.whl.metadata (3.9 kB)
  Using cached trec_car_tools-2.6-py3-none-any.whl.metadata


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\vnintern02\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [23]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('namdp-ptit/ViRanker',
                        use_fp16=True)

C:\Users\vnintern02\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vnintern02\.cache\huggingface\hub\models--namdp-ptit--ViRanker. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is ena

In [44]:
def rerank_documents(question: str, docs: List[Document], reranker : FlagReranker ,top_k = 5) -> list[Document]:
    pairs = [(question, doc.page_content) for doc in docs]
    scores = reranker.compute_score(pairs)
    doc_score_pairs = list(zip(docs, scores))
    doc_score_pairs.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, score in doc_score_pairs[:top_k]]
reranked = rerank_documents(question, retrieved_docs, reranker, top_k=5)
for i in range(len(reranked)):
    display(Markdown(reranked[i].page_content))

! [ Công ty TNHH DENSO Việt Nam - Việc chụp ảnh trong Công ty phải được thực hiện bằng máy ảnh của Công ty với mục đích công việc vàđược phê duyệt với từng mức độ bảo mật ( đeo băng tay vải màu cam và có dòng chữ “ Camera ” ) ; - Ảnh chụp trong Công ty được coi là tài sản thông tin của Công ty . Do vậy , cầm mọi hình thức phân phối ảnh chụp trong Công ty vàảnh chụp cá nhân mặc đồng phục DENSO ra bên ngoài , bao gồm cả trang mạng xã hội nếu chưa được phê duyệt bởi các cấp có thâm quyên ; - Công ty thực hiện đào tạo người lao động về hoạt động bảo vệ tài sản , bí mật kinh doanh công nghệ của Công ty . 3 . Xử lí kỷ luật Đối với những hành vi xâm phạm tài sản , bí mat kinh doanh công nghệ của Công ty , tùy theo lỗi , mức độ nghiêm trọng về hậu quả mà có thể bị xử lý kỷ luật theo điều 39 chương 7 hoặc Công ty khởi kiện theo quy định của pháp luật . CHƯƠNG 4 - NHIỆM VỤ LÀM VIỆC Phần 1 : Giờ làm việc Điều 11 : Giờ làm việc cơ bản 1 . Lịch làm việc hàng năm ( bao gồm ngày nghỉ tuần ) được quy định cụ thể và thông báo cho người lao động vào đầu năm sau khi tham khảo ý kiến của tổ chức đại diện người lao động . 2 . Đối với nhân viên khối văn phòng : 2.1 Giờ làm việc bình thường không quá 10 giờ / ngày và 48 giờ / tuần . 2.2 Ngày nghỉ hằng tuần của nhân viên là Thứ Bảy , Chủ Nhật 2.3 Giờ làm việc như sau : GIỜ LÀM VIỆC THỰC TẾ NGHỈ NGẮN ( 8 Giờ 35 Phút ) ( 20 phút ) h Nghỉ giải lao và so zÄ 。 Buổi sáng ïn trữ Buổi chiều Nghỉ ngắn § : 10 ~ 12 : 00 | 12 : 00 ~ 12 : 30 12 : 30 ~ 17 : 15 | 20 phút ( thời gian linh hoạt ) 3 . Đối với nhân viên khối nhà máy : 3.1 Ngày nghỉ hằng tuần của nhân viên là Chủ Nhật . 3.2 Về cơ bản , giờ của các ca làm việc sẽ như sau : Ca | Bấtđầu | Giảilaoldnl | Giải lao lần 2 | ( ; ; 1 olận4 | Kếtthúc và Ăn giữa ca Cai | 06 : 00 | 08 : 00 < 08 : 10 | 10 : 00 ~ 1030 | 12 : 15 ~ 1225 | 14 : 00 Ca2 | 14 : 00 | 16 : 00 ~ 16 : 10 | 18 : 00 ~ 18 : 30 | 20 : 15 ~ 20 : 25 | 22 : 00 Ca3 | 22 : 00 _ | 00 : 00 ~ 00 : 10 | 02 : 00 ~ 02 : 45 | 04 : 30 ~ 04 : 40 | 06 : 00 4 . Giờ nghỉăn giữa ca : ca ngày là 30 phút , ca đêm là 45 phút . ] ( # )

! [ Công ty TNHH DENSO Việt Nam 9 . - Nhân viên có thời gian làm việc trong khoảng từ 22 : 00 đến 6 : 00 sáng ngày hôm sau được hưởng phụ cấp làm ca đêm bằng 35 % tiền lương của công việc làm ban ngày . . Thời điểm bất đầu hoặc kết thúc của giờ làm việc được quy định ở trên có nghĩa làđã “ sắn sàng làm việc ” hoặc “ kết thúc công việc ” tại vị trí làm việc . Thời gian đi tới nơi làm việc , thay trang phục , giao ca ... và từ nơi làm việc về không được tính là thời gian làm việc . . Việc không thể bắt đầu công việc vào thời điểm bắt đầu giờ làm việc hoặc ca làm việc mà không có lí do chính đáng sẽ bị coi là “ Di muộn ” . Rời vị trí làm việc mE \ \ ] không được cấp trên cho phép trước thời điểm kết thúc giờ làm việc sẽ bị coi là “ Ve sớm ” . h TB hợp bất khả kháng như dịch bệnh , sự cốđiện nước , thiên tai , cháy .. người lao động có thểđược yêu cầu nghỉ hàng tuần vào bất kỳ ngày nào trong tuần , không cốđịnh như trên lịch và Công ty sẽ thông báo trước cho người lao động bằng văn bản . Giờ làm việc trong thời gian công tác nước ngoài được quy định riêng . Nàng 12 : Làm thêm giờ bai . Khi cần làm thêm giờ do yêu cầu sản xuất kinh doanh , Công ty là người thỏa thuận , đồng ý cho người lao động làm thêm giờ khi đáp ứng các điều kiện sau : - Phai được sựđồng ý của người lao động tham gia làm thêm về các nội dung sau đây : thời gian làm thêm , địa điểm làm thêm , công việc làm thêm ; - Bảo đảm tổng số giờ làm việc bình thường và số giờ làm thêm của người lao động không quá 12 h / ngay : Tổng số giờ làm thêm không quá 40 h / tháng , không quá 200 h / năm ; Khi làm thêm từ 200 h đến 300 h thì công ty sẽ thực hiện các bước theo quy định của pháp luật . . Làm thêm giờ trong trường hợp đặc biệt Công ty có quyên yêu cầu người lao động làm thêm giờ vào bắt kỳ ngày nào mà không bị giới hạn về số giờ làm thêm theo quy định và người lao động không được từ chối trong trường hợp sau đây : - Thực hiện lệnh động viên , huy động đảm bảo nhiệm vụ quốc phòng , an ninh theo quy định của pháp luật ; Thực hiện các công việc nhằm bảo vệ tính mạng con người , tài sản của cơ quan , tổ chức , cá nhân trong phòng ngừa , khắc phục hậu quả thiên tai , hỏa hoạn , dịch bệnh nguy hiểm và thảm họa , trừ trường hợp có nguy cơ ảnh hưởng đến . tính mạng , sức khỏe của người lao động theo quy định của pháp luật về an toàn , vệ sinh lao động . Người lao động làm thêm giờđược trả lương như sau : 3.1 Vào ngày làm việc bình thường : bằng 150 % của tiền lương giờ trong ngày làm việc bình thường ; 3.2 Vào ngày nghỉ tuần : bằng 200 % của tiền lương giờ trong ngày làm việc bình thường ; 3.3 Vào ngày nghỉ lễ : bằng 300 % của tiền lương giờ trong ngày làm việc bình thường . - 9 - ] ( # )

! [ Công ty TNHH DENSO Việt Nam 3 . Khi người lao động đến muộn do vấn đề giao thông công cộng ( có bằng chứng ) hoặc do những lý do khác mà công ty thấy rằng ngoài sự kiểm soát của người lao động thì họ sẽ không bị coi làđi làm muộn . Phần 3 : Đi công tác Điều 23 : Đi công tác 1 . Người lao động có thểđược yêu cầu đi công tác xa vì lý do công việc ( trừ trường hợp người lao động nữ mang thai từ tháng thứ 7 hoặc đang nuôi con dưới 12 tháng tuổi ) . 2 . Khi người lao động không thể quay về công ty vào khoảng thời gian đãđịnh vì lý do công việc hoặc hoàn cảnh không tránh khỏi , hoặc phải đi đến nơi khác , người lao động phải được cấp trên đồng ý và phải theo những thủ tục cần thiết ngay sau khi trở lại làm việc . 3 . Người lao động phải báo trước nơi nào người đó sẽở trong suốt thời gian công tác . 4 . Sau khi đi công tác về , người lao động phải đệ trình báo cáo cho cấp trên ngay lập tức nếu được yêu cầu . 5 . Người lao động được hưởng ngày nghỉđặc biệt hưởng nguyên lương sau khi đi học tập / công tác nước ngoài về theo quy định sau : - Nếu khoáđào tạo / công tác từ 3 tháng trở lên : nghỉ 3 ngày ; - Nếu khoáđào tạo / công tác từ 1 tháng đến dưới 3 tháng : nghỉ 1 ngày . 6 . Thủ tục thanh toán những khoản chỉ phíđi công tác sẽđược nêu riêng . CHƯƠNG 5 - SỨC KHỎE VÀ AN TOÀN Phần 1 : Thái độ cơ bản Điều 24 : Thái độ cơ bản 1 . Người lao động phải có thái độ hợp tác và tuân thủ những biện pháp đảm bảo sức khỏe và an toàn của Công ty 2 . Người lao động phải hết sức cố gắng để không xảy ra tai nạn lao động cho chính mình và những người khác . 3 . Người lao động phải tuân theo sự chỉ dẫn của Công ty để giữ vững và cải thiện sức khỏe . Phần 2 : An toàn Điều 25 : Trách nhiệm của Công ty Công ty có trách nhiệm thực hiện theo quy định của Bộ luật lao động và các văn bản pháp luật có liên quan khác về vấn đề an toàn và vệ sinh lao động , cụ thể như sau : 1 . Trang bịđầy đủ phương tiện bảo hộ lao động , đảm bảo an toàn lao động ; 2 . Có phương tiện / dụng cụ che chắn các bộ phận dễ gây nguy hiểm của máy móc , thiết bị trong Công ty ; 3 . Đặt các biển chỉ dẫn về an toàn ở nơi dễ thấy , dễđọc ; = lÄ = ] ( # )

! [ Công ty TNHH DENSO Việt Nam TIẾNG VIỆT NỘI QUY LAO ĐỘNG MỤC LỤC CHUONG 1 : NHŨNG QUY ĐỊNH CHUNG Trang : 4 Điều 1 : Mục đích của nội quy Trang : 4 Điều 2 : Phạm vi áp dụng Trang : 4 Điều 3 : Người lao động Trang : 4 Điều 4 : Cấp trên Trang : 4 Điều 5 : Người sử dụng lao động Trang : 4 CHƯƠNG 2 : TRẬT TỰTRONG CÔNG TY Trang : 4 Điều 6 : Thái độ cơ bản Trang : 4 Điều 7 : Chấp hành nội quy Trang : 4 Điều 8 : Ra vào công ty Trang : 5 CHƯƠNG 3 : CHUYỂN NGƯỜI LAO ĐỘNG LÀM Trang : 6 CÔNG VIỆC KHÁC VỚI HỢP ĐỒNG LAO ĐỘNG . BẢO VỆ BÍ MẬT KINH DOANH , CÔNG NGHỆ , SỞ HỮU TRÍ TUỆ CỦA CÔNG TY Điều 9 : Thay đổi công việc khác với Hợp đồng lao động Trang : 6 Điều 10 : Bảo vệ bí mật kinh doanh , công nghệ của công ty Trang : 7 CHƯƠNG 4 : NHIỆM VỤ LÀM VIỆC Trang : 8 Phần 1 : Giờ làm việc Trang : 8 Điều 11 : Giờ làm việc cơ bản Trang : 8 Điều 12 : Lam thêm giờ Trang : 9 Điều 13 : Ngoại lệ trong trường hợp khẩn cấp Trang : 10 Phần 2 : Ngày nghỉ Trang : 10 Điều 14 : Nghỉ lễ Trang : 10 Điều 15 : Nghỉ phép Trang : 10 ] ( # )

! [ Công ty TNHH DENSO Việt Nam 3.4 Người lao động làm thêm giờ vào ban đêm thì ngoài việc trả lương theo quy định tại khoản 3.1 , 3.2 và 3.3 Điều này , người lao động còn được trả thêm 20 % tiền lương theo tiền lương giờ làm việc ban ngày . Điều 13 : Ngoại lệ trong trường hợp khẩn cấp Khi có những nhu cầu tạm thời do ảnh hưởng nghiêm trọng của thiên tai , hành động phá hoại của kẻ thù , hỏa hoạn hoặc bệnh dịch mà Công ty phải giải quyết , Công ty có thể yêu cầu người lao động làm thêm giờ quá 4 tiếng / ngày . Công ty sẽ có báo cáo với cơ quan quản lý lao động cấp trên trong những trường hợp này . Phần 2 : Ngày nghỉ Điều 14 : Nghỉ lễ 1 . Người lao động được nghỉ làm việc , hưởng nguyên lương những ngày lễ sau : - Tết dương lịch : 1 ngày ( ngày 1 tháng 1 dương lịch ) - Tết âm lịch : 5 ngày - Ngày Chiến thắng : 1 ngày ( ngày 30 tháng 4 dương lịch ) - Ngày Quốc tế lao động : 1 ngày ( ngày 1 tháng 5 dương lịch ) - Ngày Quốc khánh : 2 ngày ( ngày 2 tháng 9 dương lịch và 1 ngày trước hoặc sau ngày Quốc Khánh ) - Ngày Giỗ Tổ Hùng Vương : 1 ngày ( ngày 10 tháng 3 âm lịch ) 2 . Nếu bất kỳ ngày lễ nào nói trên trùng vào ngày nghỉ tuần , người lao động sẽ được nghỉ bù vào ngày tiếp theo . 3 . Công ty sẽ thông báo Lịch làm việc cụ thể của năm tiếp theo cho toàn thể nhân viên vào cuối năm trước sau khi đã tham khảo ý kiến của Tổ chức đại diện người lao động trên cơ sở tuân thủ thời giờ làm việc , thời giờ nghỉ ngơi của Bộ luật lao động . Điều 15 : Nghỉ phép 1 . Cách tính ngày phép : 1.1 Thời gian thử việc và tập nghềđược coi là thời gian làm việc của người lao động để tính số ngày phép . 1.2 Người lao động làm đủ 12 tháng / năm sẽđược hưởng 12 ngày phép . Số ngày nghỉđược tăng theo thâm niên làm việc nhưng không vượt quá 20 ngày / năm , cụ thể như sau : s _ Cứ 4 năm được hưởng thêm 1 ngày s _ Từ năm thứ 8 , cứ 3 năm được hưởng thêm 1 ngày * _ Từ năm thứ 14 , cứ 2 năm được hưởng thêm 1 ngày * _ Từ năm thứ 20 , cứ 1 năm được hưởng thêm 1 ngày 1.3 Người lao động được tuyển dụng trong năm được hưởng số ngày phép như sau : * Nenu vào làm việc trước ngày 16 của tháng thì sẽđược tính phép . ngay từ tháng đó ; © Neu vào làm việc từ ngày 16 của tháng trởđi thì sẽđược tính phép từ tháng tiếp theo . = lỦ - ~ ] ( # )

In [68]:
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))

class MultiQuery:
    def __init__(self, retriever, llm=None, parser=LineListOutputParser()):
        self.retriever = retriever
        if llm is None:
            try: 
                llm = ChatOllama(model=MODEL_NAME, temperature=0.2)
                self.llm = llm
            except Exception as e:
                print(f"Error initializing LLM: {e}")
                raise
        else:
            self.llm = llm
        self.parser = parser
        self.prompt = PromptTemplate(
            input_variables=['question'],
            template = '''You are a helpful language model assistant. Your task is to generate multiple distinct queries that can help retrieve relevant documents from a vector database.
             By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. 
            Provide these alternative questions separated by newlines. Generate the different versions based ONLY on the language of the original question. 
            Output only the different questions in numerated lines.
            Original question: {question}'''
        )
    def generate_queries(self, question: str) -> List[str]:
        try:
            chain = self.prompt | self.llm | self.parser
            retriever = MultiQueryRetriever(
                retriever=self.retriever.as_retriever(),
                llm_chain=chain,
                parser_key='lines'
            )
        except Exception as e:
            print(f"Error generating queries: {e}")
            return [question]
        return retriever.invoke(question)
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)
multquery = MultiQuery(vn_vector_store, None, LineListOutputParser())
multquery.generate_queries('Giờ làm việc cơ bản?')

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Cấu trúc giờ làm việc tiêu chuẩn là gì?', '2. Thời gian làm việc bình thường bao gồm những thời điểm nào?', '3. Giờ làm việc cơ bản được quy định như thế nào trong luật lao động?', '4. Mô tả về lịch trình làm việc thông thường.', '5. Thông tin chi tiết về khung giờ làm việc cơ bản.']


[Document(id='46978f45-c70e-47f6-9571-b6371ec17961', metadata={'producer': 'ApeosPort 5570', 'creator': 'ApeosPort 5570', 'creationdate': '2022-07-27T14:18:40+07:00', 'source': 'true_test.pdf', 'file_path': 'true_test.pdf', 'total_pages': 48, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2022-07-27T14:18:40+07:00', 'trapped': '', 'modDate': "D:20220727141840+07'00'", 'creationDate': "D:20220727141840+07'00'", 'page': 0, 'language': 'vi'}, page_content='! [ NỘI QUY LAO ĐỘNG Công ty TNHH DENSO Việt Nam Hà Nội , tháng 07 năm 2022 ] ( # )'),
 Document(id='1af17246-3b85-4c87-bdaf-ebe3889824e7', metadata={'producer': 'ApeosPort 5570', 'creator': 'ApeosPort 5570', 'creationdate': '2022-07-27T14:18:40+07:00', 'source': 'true_test.pdf', 'file_path': 'true_test.pdf', 'total_pages': 48, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2022-07-27T14:18:40+07:00', 'trapped': '', 'modDate': "D:20220727141840+07'00

In [ ]:
llm = ChatOllama(model=MODEL_NAME, temperature=0.2)
gen_queries = MultiQueryRetriever.from_llm(
    retriever = vn_vector_store.as_retriever(), llm = llm,
)
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)
gen_queries.invoke('Giờ làm việc cơ bản?')

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are three different versions of the original question:', 'Giờ làm việc cơ bản có thể được xác định như thế nào?', 'Giờ làm việc cơ bản là gì và nó được áp dụng trong các trường hợp nào?', 'Giờ làm việc cơ bản của một người lao động cần phải đáp ứng những yêu cầu pháp lý cụ thể.']


[Document(id='31af87a0-1f20-4041-aed4-5c3c491948ca', metadata={'producer': 'ApeosPort 5570', 'creator': 'ApeosPort 5570', 'creationdate': '2022-07-27T14:18:40+07:00', 'source': 'true_test.pdf', 'file_path': 'true_test.pdf', 'total_pages': 48, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2022-07-27T14:18:40+07:00', 'trapped': '', 'modDate': "D:20220727141840+07'00'", 'creationDate': "D:20220727141840+07'00'", 'page': 47, 'language': 'vi'}, page_content='! [ ] ( # )'),
 Document(id='46978f45-c70e-47f6-9571-b6371ec17961', metadata={'producer': 'ApeosPort 5570', 'creator': 'ApeosPort 5570', 'creationdate': '2022-07-27T14:18:40+07:00', 'source': 'true_test.pdf', 'file_path': 'true_test.pdf', 'total_pages': 48, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2022-07-27T14:18:40+07:00', 'trapped': '', 'modDate': "D:20220727141840+07'00'", 'creationDate': "D:20220727141840+07'00'", 'page': 0, 'language': '

In [ ]:
class RAG:
    def __init__(self, retriever, llm = None):
        self.retriever = retriever
        if llm is None:
            try: 
                self.llm = ChatOllama(model=MODEL_NAME, temperature = 0, max_tokens = 1024)
            except Exception as e:
                logging.error(f'Error initializing LLM: {e}')
                self.llm = None
        else:
            self.llm = llm
    def router(self, question: str) -> List[Document]:
        return self.retriever.invoke(question)